In [ ]:
import os
import requests
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
from pathlib import Path
from tqdm import tqdm

# Configuration
BASE_URL = "https://langchain-ai.github.io/langgraph/"
OUTPUT_DIR = Path("LG_documentation_website_data")
OUTPUT_DIR.mkdir(exist_ok=True)

# File type extensions
PDF_EXTENSIONS = {".pdf"}
IMAGE_EXTENSIONS = {".jpg", ".jpeg", ".png", ".gif", ".bmp", ".svg", ".webp"}
YOUTUBE_DOMAINS = {"youtube.com", "youtu.be"}

# Storage
visited_urls = set()
html_pages = []
pdf_links = []
image_links = []
youtube_links = []

def is_valid_url(url):
    parsed = urlparse(url)
    return parsed.scheme in {"http", "https"}

def is_youtube_url(url):
    parsed = urlparse(url)
    return any(domain in parsed.netloc for domain in YOUTUBE_DOMAINS)

def download_file(url, folder):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        filename = urlparse(url).path.split("/")[-1]
        filepath = folder / filename
        with open(filepath, "wb") as f:
            f.write(response.content)
        return filepath
    except Exception as e:
        print(f"Failed to download {url}: {e}")
        return None

def crawl(url):
    to_visit = [url]
    while to_visit:
        current_url = to_visit.pop()
        if current_url in visited_urls:
            continue
        visited_urls.add(current_url)

        try:
            response = requests.get(current_url, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")
            html_pages.append(current_url)

            # Save HTML content
            parsed_url = urlparse(current_url)
            filename = parsed_url.path.strip("/").replace("/", "_") or "index"
            filepath = OUTPUT_DIR / f"{filename}.html"
            with open(filepath, "w", encoding="utf-8") as f:
                f.write(response.text)

            # Extract and categorize links
            for tag in soup.find_all("a", href=True):
                href = tag["href"]
                full_url = urljoin(current_url, href)
                if not is_valid_url(full_url):
                    continue
                parsed_href = urlparse(full_url)
                ext = os.path.splitext(parsed_href.path)[1].lower()

                if full_url.startswith(BASE_URL) and full_url not in visited_urls:
                    to_visit.append(full_url)
                elif ext in PDF_EXTENSIONS:
                    pdf_links.append(full_url)
                elif ext in IMAGE_EXTENSIONS:
                    image_links.append(full_url)
                elif is_youtube_url(full_url):
                    youtube_links.append(full_url)
        except Exception as e:
            print(f"Failed to fetch {current_url}: {e}")

# Start crawling
crawl(BASE_URL)

# Download PDFs and images
pdf_folder = OUTPUT_DIR / "pdfs"
image_folder = OUTPUT_DIR / "images"
pdf_folder.mkdir(exist_ok=True)
image_folder.mkdir(exist_ok=True)

for pdf_url in tqdm(pdf_links, desc="Downloading PDFs"):
    download_file(pdf_url, pdf_folder)

for image_url in tqdm(image_links, desc="Downloading Images"):
    download_file(image_url, image_folder)

# Save YouTube links
youtube_file = OUTPUT_DIR / "youtube_links.txt"
with open(youtube_file, "w", encoding="utf-8") as f:
    for link in youtube_links:
        f.write(f"{link}\n")

print("Crawling and downloading completed.")


Failed to fetch https://langchain-ai.github.io/langgraph/agentic_concepts/: 404 Client Error: Not Found for url: https://langchain-ai.github.io/langgraph/agentic_concepts/
Failed to fetch https://langchain-ai.github.io/langgraph/high_level/: 404 Client Error: Not Found for url: https://langchain-ai.github.io/langgraph/high_level/
Failed to fetch https://langchain-ai.github.io/langgraph/streaming/: 404 Client Error: Not Found for url: https://langchain-ai.github.io/langgraph/streaming/
Failed to fetch https://langchain-ai.github.io/langgraph/multi_agent/: 404 Client Error: Not Found for url: https://langchain-ai.github.io/langgraph/multi_agent/
Failed to fetch https://langchain-ai.github.io/langgraph/breakpoints/: 404 Client Error: Not Found for url: https://langchain-ai.github.io/langgraph/breakpoints/
Failed to fetch https://langchain-ai.github.io/langgraph/human_in_the_loop/: 404 Client Error: Not Found for url: https://langchain-ai.github.io/langgraph/human_in_the_loop/
Failed to fe

In [ ]:
# !pip install -q tldextract unstructured

In [ ]:
# from langchain.document_loaders import UnstructuredHTMLLoader

# loader = UnstructuredHTMLLoader("chaicode_documentation_website_data/contribute_starter-kit_authoring-content.html")
# docs   = loader.load()    
# docs

In [ ]:
import glob
from pathlib import Path
from langchain.document_loaders import UnstructuredHTMLLoader

# 1. Find all HTML files under the folder
html_dir = Path("LG_documentation_website_data") # LG_documentation_website_data
html_paths = sorted(html_dir.glob("**/*.html"))

# 2. Load & clean each page
all_docs = []
for path in html_paths:
    loader = UnstructuredHTMLLoader(str(path))
    docs = loader.load()  # returns a list of Document(page_content, metadata)
    for d in docs:
        # Attach a more precise source reference if you like
        d.metadata["source_path"] = str(path)
    all_docs.extend(docs)

# 3. Inspect how many documents you got
print(f"Loaded {len(all_docs)} document(s) from {len(html_paths)} HTML file(s).")

# 4. (Optional) Save out to disk for later reuse
import json
out = [
    {"text": doc.page_content, "source": doc.metadata.get("source_path")}
    for doc in all_docs
]
with open("LG_docs_html.json", "w", encoding="utf-8") as f:
    json.dump(out, f, indent=2, ensure_ascii=False)

# Now `all_docs` holds the full cleaned text of every HTML page,
# ready for chunking, embedding, and retrieval in your LLM pipeline.


Loaded 50 document(s) from 50 HTML file(s).


In [ ]:
import json
from langchain.schema import Document  # if that errors, try from langchain.docstore.document import Document

# 1. Load your JSON dump
with open("LG_docs_html.json", "r", encoding="utf-8") as f:
    items = json.load(f)

# 2. Reconstruct Documents
all_docs_reconstruct = [
    Document(
        page_content=item["text"],
        metadata={"source_path": item["source"]}
    )
    for item in items
]

# 3. Verify
print(f"Reconstructed {len(all_docs_reconstruct)} Document objects.")


Reconstructed 50 Document objects.


In [ ]:
all_docs[:2]

[Document(metadata={'source': 'adk_documentation_website_data/adk-docs.html', 'source_path': 'adk_documentation_website_data/adk-docs.html'}, page_content='Agent Development Kit Logo\n\nAgent Development Kit\n\nWhat is Agent Development Kit?¶\n\nAgent Development Kit (ADK) is a flexible and modular framework for developing and deploying AI agents. While optimized for Gemini and the Google ecosystem, ADK is model-agnostic, deployment-agnostic, and is built for compatibility with other frameworks. ADK was designed to make agent development feel more like software development, to make it easier for developers to create, deploy, and orchestrate agentic architectures that range from simple tasks to complex workflows.\n\nGet started: pip install google-adk\n\nQuickstart Tutorials Sample Agents API Reference Contribute ❤️\n\nLearn more¶\n\nFlexible Orchestration\n\nDefine workflows using workflow agents (Sequential, Parallel, Loop) for predictable pipelines, or leverage LLM-driven dynamic rou

In [ ]:
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables from the .env file
load_dotenv()
gemini_api_key = os.getenv("GOOGLE_API_KEY")
# print(gemini_api_key)
gemini_llm  = ChatGoogleGenerativeAI(model="gemini-2.0-flash")



In [ ]:
# # from langchain_groq.chat_models import ChatGroq
# # llm_2 = ChatGroq(model="llama3-70b-8192", temperature=0)
# # llm_1 = ChatGroq(model="llama3-70b-8192", temperature=0)

from langchain_groq.chat_models import ChatGroq

# llms = [
#     ChatGroq(model="llama-3.3-70b-versatile", temperature=0),
#     ChatGroq(model="gemma2-9b-it", temperature=0),
#     ChatGroq(model="deepseek-r1-distill-llama-70b", temperature=0),
#     ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0),
#     ChatGroq(model="meta-llama/llama-4-maverick-17b-128e-instruct", temperature=0),
#     ChatGroq(model="llama-3.1-8b-instant", temperature=0),

# ]


In [ ]:
# llm = ChatOllama(model="llama3.2:3b-text-q8_0", temperature=0)
import time
import gc
from langchain.prompts import PromptTemplate
from langchain_groq.chat_models import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm
from langchain_groq.chat_models import ChatGroq

In [ ]:


# Optional: Torch GPU memory cleanup
try:
    import torch
    def clear_gpu_cache():
        torch.cuda.empty_cache()
except ImportError:
    def clear_gpu_cache():
        pass  # Torch not installed, ignore

# Time helper
def timestamp():
    return time.strftime("[%H:%M:%S]")

gemini_llm  = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
def fallback_chain():
    chain = contextual_prompt | gemini_llm | StrOutputParser()
    return chain

# 1. Prompt template
contextual_prompt = PromptTemplate(
    template="""
<document>
{whole_document}
</document>

Here is the chunk we want to situate within the whole document:
<chunk>
{chunk}
</chunk>

Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk. Answer only with the succinct context and nothing else.
""",
    input_variables=["whole_document", "chunk"]
)

# 2. List of Groq model names
model_names = [
    "llama-3.3-70b-versatile",
    "gemma2-9b-it",
    "deepseek-r1-distill-llama-70b",
    "meta-llama/llama-4-scout-17b-16e-instruct",
    "meta-llama/llama-4-maverick-17b-128e-instruct",
    "llama-3.1-8b-instant",
    "deepseek-r1-distill-llama-70b",
    "llama-3.3-70b-versatile",
    "llama3-70b-8192",
    "llama3-8b-8192",
    "meta-llama/llama-4-scout-17b-16e-instruct"
]

# 3. Chunking function
def get_contextual_chunks(documents, chunk_size=1500, chunk_overlap=300):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    enriched_chunks = []

    print(f"{timestamp()} 🚀 Starting contextual chunking for {len(documents)} documents...")

    chunk_counter = 0

    for doc_idx, doc in enumerate(tqdm(documents)):
        print(f"{timestamp()} 🗂️ Starting document {doc_idx + 1} → Source: {doc.metadata.get('source_path', 'unknown')}")

        chunks = splitter.split_text(doc.page_content)

        for i, chunk in enumerate(chunks):
            try:
                model_index = i % len(model_names)  # Rotate among models
                model_name = model_names[model_index]
                print(f"{timestamp()} 🔄 Using model {model_index}: {model_name}")

                # Instantiate ChatGroq with the selected model
                llm = ChatGroq(model=model_name, temperature=0)
                chain = contextual_prompt | llm | StrOutputParser()

                try:
                    context_summary = chain.invoke({
                        "whole_document": doc.page_content,
                        "chunk": chunk
                    })
                except Exception as groq_error:
                    print(f"{timestamp()} ❌ Groq failed for chunk {chunk_counter}: {groq_error}")
                    try:
                        print(f"{timestamp()} 🔁 Retrying with Gemini...")
                        context_summary = fallback_chain().invoke({
                            "whole_document": doc.page_content,
                            "chunk": chunk
                        })
                    except Exception as gemini_error:
                        print(f"{timestamp()} 💥 Gemini also failed: {gemini_error}")
                        continue  # Skip this chunk

                # Cleanup
                del llm
                del chain
                gc.collect()
                clear_gpu_cache()

                enriched_doc = Document(
                    page_content=f"{context_summary}\n\n{chunk}",
                    metadata={**doc.metadata, "context_summary": context_summary}
                )
                enriched_chunks.append(enriched_doc)

                chunk_counter += 1
                print(f"{timestamp()} 🧩 Processed {chunk_counter} chunks so far...")
                time.sleep(3)

            except Exception as e:
                print(f"{timestamp()} ⚠️ Failed on chunk in {doc.metadata.get('source', 'unknown')}: {e}")
                continue

        print(f"{timestamp()} ✅ Finished document {doc_idx + 1} ({len(chunks)} chunks)")

    print(f"{timestamp()} 🎉 Done! Total enriched chunks created: {len(enriched_chunks)}")
    return enriched_chunks


In [ ]:



# splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)
# chunks = splitter.split_text(all_docs[2].page_content)
# chunks[1]

In [ ]:
# Load HTML page content into Document objects (already done)
# docs = [Document(page_content="...your html...", metadata={"source": "file1.html"})]

# Run contextual chunking
contextual_chunks = get_contextual_chunks(all_docs)

# contextual_chunks


[22:35:05] 🚀 Starting contextual chunking for 50 documents...


  0%|          | 0/50 [00:00<?, ?it/s]

[22:35:05] 🗂️ Starting document 1 → Source: adk_documentation_website_data/adk-docs.html
[22:35:05] 🔄 Using model 0: llama-3.3-70b-versatile
[22:35:05] 🧩 Processed 1 chunks so far...
[22:35:08] 🔄 Using model 1: gemma2-9b-it
[22:35:09] 🧩 Processed 2 chunks so far...


  2%|▏         | 1/50 [00:07<06:08,  7.52s/it]

[22:35:12] ✅ Finished document 1 (2 chunks)
[22:35:12] 🗂️ Starting document 2 → Source: adk_documentation_website_data/adk-docs_agents.html
[22:35:12] 🔄 Using model 0: llama-3.3-70b-versatile
[22:35:13] 🧩 Processed 3 chunks so far...
[22:35:16] 🔄 Using model 1: gemma2-9b-it
[22:35:17] 🧩 Processed 4 chunks so far...
[22:35:20] 🔄 Using model 2: deepseek-r1-distill-llama-70b
[22:35:22] 🧩 Processed 5 chunks so far...


...

 98%|█████████▊| 49/50 [33:57<00:30, 30.99s/it]

[23:09:02] ✅ Finished document 49 (1 chunks)
[23:09:02] 🗂️ Starting document 50 → Source: adk_documentation_website_data/adk-docs_tutorials_agent-team.html
[23:09:02] 🔄 Using model 0: llama-3.3-70b-versatile
[23:09:02] ❌ Groq failed for chunk 405: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01j0yrgwx6fghspw04qqs0jfsf` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99344, Requested 25530. Please try again in 5h58m10.607s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
[23:09:02] 🔁 Retrying with Gemini...
[23:09:04] 🧩 Processed 406 chunks so far...
[23:09:07] 🔄 Using model 1: gemma2-9b-it
[23:09:07] ❌ Groq failed for chunk 406: Error code: 413 - {'error': {'message': 'Request too large for model `gemma2-9b-it` in organization `org_01j0yrgwx6fghspw04qqs0jfsf` service tier `on_demand` on tokens per minut

100%|██████████| 50/50 [43:27<00:00, 52.16s/it] 

[23:18:33] ✅ Finished document 50 (87 chunks)
[23:18:33] 🎉 Done! Total enriched chunks created: 492


[Document(metadata={'source': 'adk_documentation_website_data/adk-docs.html', 'source_path': 'adk_documentation_website_data/adk-docs.html', 'context_summary': 'Introduction to Agent Development Kit and its key features.'}, page_content='Introduction to Agent Development Kit and its key features.\n\nAgent Development Kit Logo\n\nAgent Development Kit\n\nWhat is Agent Development Kit?¶\n\nAgent Development Kit (ADK) is a flexible and modular framework for developing and deploying AI agents. While optimized for Gemini and the Google ecosystem, ADK is model-agnostic, deployment-agnostic, and is built for compatibility with other frameworks. ADK was designed to make agent development feel more like software development, to make it easier for developers to create, deploy, and orchestrate agentic architectures that range from simple tasks to complex workflows.\n\nGet started: pip install google-adk\n\nQuickstart Tutorials Sample Agents API Reference Contribute ❤️\n\nLearn more¶\n\nFlexible O

In [ ]:
# # contextual_chunks
import pickle

with open("enriched_chunks_adk.pkl", "wb") as f:
    pickle.dump(contextual_chunks, f)


# with open("enriched_chunks_adk.pkl", "rb") as f:
#     enriched_chunks = pickle.load(f)

In [ ]:
# type(enriched_chunks)

In [ ]:
type(contextual_chunks)

list

## Embedding 

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name = '/mnt/c/Users/Nikhil/Desktop/chai_code_genai/RAG_on_chaicode_docs_website/models/all-mpnet-base-v2',
                                        
                                        model_kwargs={'device': 'cuda'})


/tmp/ipykernel_652294/1807814223.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name = '/mnt/c/Users/Nikhil/Desktop/chai_code_genai/RAG_on_chaicode_docs_website/models/all-mpnet-base-v2',


In [ ]:
from langchain.vectorstores import Chroma

# Assume enriched_chunks is your list of Document objects (with context + metadata)
vectorstore = Chroma.from_documents(
    documents=contextual_chunks,
    embedding=embedding_model,                                   # Embeddings obj
    collection_name="lg_docs",
    persist_directory="contextual_chunked_LG_docs"  # this will save it locally
)


# Optional: persist to disk
vectorstore.persist()

/tmp/ipykernel_615538/941654604.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
# Load it anytime
vectorstore = Chroma(
    persist_directory="contextual_chunked_LG_docs",
    embedding_function=embedding_model
)

query = "How to push interactions to DB ?"
# results = vectorstore.similarity_search(query, k=3)
results = vectorstore.max_marginal_relevance_search(query, k=5, fetch_k=10)

for doc in results:
    print(doc.metadata)
    print(doc.page_content)
    print("-" * 50)

/tmp/ipykernel_615538/649143909.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


{'context_summary': 'This section introduces the Session object in the ADK, explaining its purpose in tracking conversation threads and detailing its key properties like ID, app name, user ID, history, state, and activity tracking.', 'source': 'adk_documentation_website_data/adk-docs_sessions_session.html', 'source_path': 'adk_documentation_website_data/adk-docs_sessions_session.html'}
This section introduces the Session object in the ADK, explaining its purpose in tracking conversation threads and detailing its key properties like ID, app name, user ID, history, state, and activity tracking.

Session: Tracking Individual Conversations¶

Following our Introduction, let's dive into the Session. Think back to the idea of a "conversation thread." Just like you wouldn't start every text message from scratch, agents need context from the ongoing interaction. Session is the ADK object designed specifically to track and manage these individual conversation threads.

The Session Object¶

When 

In [ ]:
from langgraph.graph import StateGraph, END
# from langgraph.graph.schema import BaseState
from typing import TypedDict, List

from langchain.vectorstores import Chroma
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
# from langchain_community.chat_models import ChatGroq  # or ChatOllama, Gemini

# -------------------------------
# 1. Define the Shared State
# -------------------------------
class GraphState(TypedDict):
    question: str
    context_docs: List  # list of Documents
    answer: str

# embedding_model = HuggingFaceEmbeddings(model_name = '/mnt/c/Users/Nikhil/Desktop/chai_code_genai/RAG_on_chaicode_docs_website/models/all-mpnet-base-v2',
#                                         model_kwargs={'device': 'cuda'})
# -------------------------------
# 2. Define Node Functions
# -------------------------------

def retrieval_node(state: GraphState):
    query = state['question']
    vectorstore = Chroma(
        persist_directory="contextual_chunked_LG_docs",
        embedding_function=embedding_model)
    # retriever = vectorstore.as_retriever()

    docs = vectorstore.max_marginal_relevance_search(query, k=5, fetch_k=10)
    return {"context_docs": docs}

def llm_answer_node(state: GraphState):
    context = "\n\n".join(doc.page_content for doc in state['context_docs'])

    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="""
You are a helpful assistant answering user queries based on provided documentation context.

<context>
{context}
</context>

Question: {question}
Answer:"""
    )

    llm = ChatGroq(model="llama3-70b-8192", temperature=0)
    chain = prompt | llm
    result = chain.invoke({
        "context": context,
        "question": state['question']
    })

    return {"answer": result}

# -------------------------------
# 3. Build the Graph
# -------------------------------
graph = StateGraph(GraphState)

# Register nodes using function names
graph.add_node("retrieval", retrieval_node)
graph.add_node("generate", llm_answer_node)

# Set up flow
graph.set_entry_point("retrieval")
graph.add_edge("retrieval", "generate")
graph.set_finish_point("generate")

# Compile the graph
rag_graph = graph.compile()

# -------------------------------
# 4. Run It
# -------------------------------
# response = rag_graph.invoke({"question": "How can i maximize my learning"})

# print("🧠 Final Answer:\n", response["answer"])


In [ ]:
question = "How to push interactions to DB ? "
response = rag_graph.invoke({"question": question})
print(response['answer'].content)

/tmp/ipykernel_652294/3981346911.py:26: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


The provided documentation focuses on the event processing flow within the Agent Development Kit (ADK) and emphasizes the importance of using the `append_event` method for updating session state to ensure proper persistence and thread safety. It also details how to create new sessions and send queries using cURL commands. However, it **does not** include specific code snippets or instructions on how to directly push interactions to a database.

The documentation explains that the `SessionService` handles persisting events to history, and services like `DatabaseSessionService` rely on the `append_event` flow to trigger saving. Therefore, to ensure interactions are saved to the database, you should:

1.  **Ensure you are using a `SessionService` that supports database persistence** (e.g., `DatabaseSessionService`).
2.  **Make sure all state updates are performed through `EventActions.state_delta` within the `append_event` flow.**  Avoid directly modifying `session.state`.


In [11]:
question = "How to push interactions to DB ? Give me the concise Code "
response = rag_graph.invoke({"question": question})
print(response['answer'].content)

Based on the provided documentation, it seems that the interactions are pushed to the database through the `append_event` method of the `SessionService`. This method is called by the `Runner` when it receives an event, and it persists the event to the session's history.

Here is a concise code snippet that demonstrates how to push interactions to the database:
```
runner = ...  # assume you have a runner instance
event = ...  # assume you have an event instance

# Send the event to the SessionService
runner.send(event)

# The SessionService will call append_event, which will persist the event to the database
```
Note that the `append_event` method is not explicitly called in this code snippet, as it is part of the internal processing flow of the `SessionService`. By sending the event to the `Runner`, it will be processed and persisted to the database automatically.


In [7]:
question = "Create a custom Agent"
response = rag_graph.invoke({"question": question})
print(response['answer'].content)

Based on the documentation, here's how you can create a custom agent using the Agent Development Kit (ADK):

1.  **Understand the Need for Customization:** Custom agents are useful when you need conditional logic or branching based on the outcome of sub-agent tasks, which isn't available in standard workflow agents. The `StoryFlowAgent` example in the documentation shows a scenario where the story is regenerated if the tone check fails.

2.  **Define Your Custom Agent Class:** Inherit from `BaseAgent` to create your custom agent.

    ```python
    from google_adk.core.agent import BaseAgent
    from google_adk.core.llm.llm_agent import LlmAgent  # Import other agent types as needed

    class StoryFlowAgent(BaseAgent):
        """
        Custom agent for a story generation and refinement workflow.

        This agent orchestrates a sequence of LLM agents to generate a story,
        critique it, revise it, check grammar and tone, and potentially
        regenerate the story if the to

In [ ]:
question = "Create a custom Agent"
response = rag_graph.invoke({"question": question})
print(response['answer'].content)

To create a custom agent, you can follow these steps:

1. Define a class that inherits from `BaseAgent`.
2. In the `__init__` method, store the necessary sub-agents as instance attributes and tell the `BaseAgent` framework about the top-level agents this custom agent will directly orchestrate.

Here is an example of a custom agent, `StoryFlowAgent`, that generates and refines a story through multiple sub-agents:
```
class StoryFlowAgent(BaseAgent):
    """
    Custom agent for a story generation and refinement workflow.

    This agent orchestrates a sequence of LLM agents to generate a story,
    critique it, revise it, check grammar and tone, and potentially
    regenerate the story if the tone is negative.
    """

    # --- Field Declarations for Pydantic ---
    # Declare the agents passed during initialization as class attributes with type hints
    story_generator: LlmAgent
    critic: LlmAgent
    reviser: LlmAgent
    grammar_check: LlmAgent
    tone_check: LlmAgent

    loop_

In [13]:
question = "Create a multiagent Agent system running parallel"
response = rag_graph.invoke({"question": question})
print(response['answer'].content)

To create a multi-agent system running parallel, you can use the ParallelAgent in the Agent Development Kit (ADK). Here's a step-by-step guide:

1. **Create a new project structure**: Create a new folder for your project, and inside it, create another folder for your agent. For example, `parent_folder/multi_agent_system/`.

2. **Create an `__init__.py` file**: In the `multi_agent_system` folder, create an `__init__.py` file with the following content: `from . import agent`.

3. **Create an `agent.py` file**: In the same folder, create an `agent.py` file where you'll define your agents.

4. **Define your sub-agents**: In the `agent.py` file, define multiple sub-agents that will run in parallel. These sub-agents can be instances of `BaseAgent` or any other type of agent.

5. **Create a ParallelAgent**: Create a `ParallelAgent` instance and pass the list of sub-agents to it.

6. **Implement the `run_async` method**: In the `ParallelAgent`, implement the `run_async` method to initiate the 

In [15]:
question = "How do i save interactions in the DB ? how to use append_event give me the code"
response = rag_graph.invoke({"question": question})
print(response['answer'].content)

To save interactions in the database, you need to use the `append_event` method provided by the `SessionService`. This method ensures that the event is recorded in the session's event history and triggers the persistence of the session state.

Here's an example of how to use `append_event`:
```python
# Assume you have a SessionService instance and an Event instance
session_service = InMemorySessionService()  # or DatabaseSessionService()
event = Event(...)  # create an Event instance with the necessary data

# Append the event to the session
session_service.append_event(session_id, event)
```
In this example, `session_id` is the unique identifier of the conversation thread, and `event` is the Event instance that you want to save.

By calling `append_event`, you ensure that the event is added to the session's event history, and the session state is updated accordingly. The `SessionService` will take care of persisting the session state to the database.

Note that you should use `Databas